## Добрый день, дорогие друзья.

Этот ноутбук содержит код для статьи "Извлекаем признаки из временных рядов" на портале NewTechAudit.  
Разделы тетрадки повторяют содержание текста.  

In [ ]:
# загружаем библиотеки
import numpy as np 
import pandas as pd 
from datetime import datetime as dt
from tqdm import tqdm
import gc

from scipy import stats
from scipy.stats import kurtosis
from scipy.signal import find_peaks

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 900)
pd.set_option('display.max_rows', 900)

import tsfresh
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import settings

# удаляем нежелательные оповещения
import warnings
warnings.simplefilter('ignore')

### Курс молодого бойца

#### Загрузка данных

In [ ]:
# Классический метод
df = pd.read_csv('../input/residential-power-usage-3years-data-timeseries/power_usage_2016_to_2020.csv')
df.head()

In [ ]:
# Теперь дату сохраняем в столбце индекса, сразу переводим индекс в datetime
df = pd.read_csv('../input/residential-power-usage-3years-data-timeseries/power_usage_2016_to_2020.csv', 
                 index_col=[0], 
                 parse_dates=[0])

df.drop(['day_of_week', 'notes'], axis=1, inplace=True)
df.head()

In [ ]:
df.info()

#### Ресемплирование

In [ ]:
# Переведем часовые наблюдения за расходом электроэнергии в недельные
df.resample('w').sum().head()

#### Проверка на монотонность

In [ ]:
print('монотонны' if df.index.is_monotonic else 'проверь данные')

In [ ]:
print('уникальны' if df.index.is_unique else 'проверь данные')

В нашем случае временные данные не монотонны, но уникальный. Попробуем отсортировать данные и вновь проверить на монотонность.

In [ ]:
df.sort_index(inplace=True)
df.index.is_monotonic

In [ ]:
# Посмотрим на начальную и конечную даты наблюдений

print('Дата начала наблюдений: {}'.format(df.index.min()))
print('Дата окончания наблюдений: {}'.format(df.index.max()))
print('Временной отрезок: {}'.format(df.index.max() - df.index.min()))

Мы можем исследовать данные за 1796 дней и 23 часа. Это чуть менее 5 лет. Хорошие данные для учебного примера.  
Наблюдения в примере приведены в виде суммы затраченной электроэнергии за каждый час.

In [ ]:
# Для удобства обработки данных переименуем колонку затраченной энергии

df.columns = ['value']
df.head()

### День, окно, лаг и разность

In [ ]:
# Выделяем простые временные признаки

df['year'] = df.index.year
df['quarter'] = df.index.quarter
df['month'] = df.index.month
df['week'] = df.index.weekofyear
df['day'] = df.index.day
df['dayofweek'] = df.index.dayofweek

df.head()

In [ ]:
# Скользящее окно (скользящее среднее)

df['rolling_window'] = df['value'].rolling(6).mean()
df.head(15)

In [ ]:
# Скользящее окно для расчета эксцессов

df['kurtosis'] = df['value'].rolling(6).apply(lambda x: kurtosis(x))


In [ ]:
df.head(10)

In [ ]:
# Скользящее окно для расчета пиков

df['peaks'] = df['value'].rolling(6).apply(lambda x: len(find_peaks(x)[0]))

In [ ]:
# Расширяющееся окно

df['expanding_window'] = df['value'].expanding(3).mean()
df.head(15)

In [ ]:
print('Финальное значение раскрывающегося окна {}'.format(df.tail(1)['expanding_window'].values[0].round(5)))
print('Среднее значение по столбцу value {}'.format(round(df['value'].mean(), 5)))

In [ ]:
### Создаем лаги
df['lag_-1'] = df['value'].shift(-1)
df['lag_1'] = df['value'].shift(1)
df.head()

In [ ]:
try:
    df.drop('lag_-1', axis=1, inplace=True)
except:
    pass

# Пробегаемся циклом
for i in range(1, 12):
    df['lag_' + str(i)] = df['value'].shift(i)
    
df.head(10)

In [ ]:
# Скользящее окно на седьмом лаге
# Приведен в качестве примера

df['lag_7_mean'] = df['lag_7'].rolling(7).mean()

In [ ]:
# некоторые столбцы во временном датасете (df_temp) не показаны 
df['diff'] = df['value'].diff()

df_temp = df[['value', 'year', 'quarter', 'month', 'week', 'day', 'dayofweek',
             'rolling_window', 'expanding_window', 'diff',
             'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7']]
df_temp.head(10)

In [ ]:
df.dropna(inplace=True)

In [ ]:
### Кодируем массивные признаки

df['year'] = df['year'].apply(lambda x: 2022 - x)

df.head()

## Воскресенье - радостный день

Добавим признак, является ли день выходным. Строка получит 0, если день рабочий, и 1, если выходной. Учитываются государственные праздники.

In [ ]:
 def is_dayoff(row):   
    '''
    Возвращает 1 если текущий день - выходной или праздничный, 0 - если будний.
    Применяется построчно к DataFrame.
    Например, df['dayoff'] = df.apply(is_dayoff, axis=1)
    
    Parameters
    ----------
    row - строка pandas DataFrame
    
    Return
    ------
    row - новая запись в pandas DataFrame
    '''
    
    if row['month'] == 1 and row['day'] == 1:
        return 1
    # примерно рассчитаем День Мартина Лютера Кинга
    elif row['month'] == 1 and row['dayofweek'] == 0 and row['week'] == 8:
        return 1
    # примерно рассчитаем Президентский день
    elif row['month'] == 2 and row['dayofweek'] == 0 and row['week'] == 10:
        return 1
    # примерно рассчитаем День памяти
    elif row['month'] == 5 and row['dayofweek'] == 0 and row['week'] == 22:
        return 1  
    # День Независимости
    elif row['month'] == 7 and row['day'] == 4:
        return 1
    # примерно рассчитаем День труда
    elif row['month'] == 9 and row['dayofweek'] == 0 and row['week'] == 37:
        return 1  
    # примерно рассчитаем День Колумба
    elif row['month'] == 9 and row['dayofweek'] == 0 and row['week'] == 42:
        return 1 
    # День Ветеранов
    elif row['month'] == 11 and row['day'] == 11:
        return 1
    # примерно рассчитаем День Благодарения
    elif row['month'] == 11 and row['dayofweek'] == 3 and row['week'] == 48:
        return 1 
    # Рождество
    elif row['month'] == 12 and row['day'] == 25:
        return 1
    
    # Обычные выходные
    elif row['dayofweek'] >= 5:
        return 1
    else:
        return 0

In [ ]:
df['is_dayoff'] = df.apply(is_dayoff, axis=1)
df.head()

### Автоматизация

In [ ]:
import featuretools as ft

# Загружаем первичные данные, но не переводим дату в индекс 

df_fe = pd.read_csv('../input/residential-power-usage-3years-data-timeseries/power_usage_2016_to_2020.csv', 
                 parse_dates=[0])

df_fe.drop(['day_of_week', 'notes'], axis=1, inplace=True)
df_fe.head()

In [ ]:
es = ft.EntitySet(id = 'data')
es.entity_from_dataframe(entity_id = 'timeseries', 
                         dataframe = df_fe, 
                         index='index')

primitives = ['day', 'is_weekend', 'week', 'year', 'minute', 'hour', 'weekday', 'cum_mean']    # Список фичей для генерации

# Запускаем создание новых признаков
feature_matrix, feature_defs = ft.dfs(entityset = es,                                          # Какой EntiteSet обрабатываем
                                      target_entity = 'timeseries',                            # Какой датафрейм изменяем
                                      trans_primitives = primitives,                           # Какие фичи создаем
                                      verbose=1)  

In [ ]:
feature_matrix.head()

In [ ]:
# Список всех доступных примитивных фичей
#ft.primitives.list_primitives()

### Бонус для тех, кто дочитал до конца
Полезные ссылки

https://towardsdatascience.com/feature-engineering-on-time-series-data-transforming-signal-data-of-a-smartphone-accelerometer-for-72cbe34b8a60  
https://otus.ru/nest/post/1024/  
https://tsfresh.readthedocs.io/en/latest/text/forecasting.htmlhttps://tsfresh.readthedocs.io/en/latest/text/forecasting.html